<a href="https://colab.research.google.com/github/Smita569/E-commerce_Marketing_and_sales/blob/main/E_commerce_Marketing_and_sales_Business_Case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement 📈

The e-commerce company aims to leverage **data-driven insights** 🧠 to enhance:

*   Customer Acquisition 🤝
*   Customer Retention ✅
*   Revenue Optimization 💰

The following analysis will be conducted to:

*   Understand key business trends 📊
*   Improve decision-making 🤔

In [16]:
# Upload csv files into dataframe

df_customerData = pd.read_csv('E-commerce_Marketing_and_sales/CustomersData.csv')
df_discountCoupon = pd.read_csv('E-commerce_Marketing_and_sales/Discount_Coupon.csv')
df_marketingSpend = pd.read_csv('E-commerce_Marketing_and_sales/Marketing_Spend.csv')
df_onlineSales = pd.read_csv('E-commerce_Marketing_and_sales/Online_Sales.csv')
df_taxAmount=pd.read_csv('E-commerce_Marketing_and_sales/Tax_amount.csv')


In [ ]:
# Import libraries

import pandas as pd
import numpy as np

In [19]:
# Read customer data
df_customerData.head()

,CustomerID,Gender,Location,Tenure_Months
0,17850,M,Chicago,12
1,13047,M,California,43
2,12583,M,Chicago,33
3,13748,F,California,30
4,15100,M,California,49


In [20]:
# Read discount coupon data
df_discountCoupon.head()

,Month,Product_Category,Coupon_Code,Discount_pct
0,Jan,Apparel,SALE10,10
1,Feb,Apparel,SALE20,20
2,Mar,Apparel,SALE30,30
3,Jan,Nest-USA,ELEC10,10
4,Feb,Nest-USA,ELEC20,20


In [22]:
# Read marketing spend data
df_marketingSpend.head()

,Date,Offline_Spend,Online_Spend
0,1/1/2019,4500,2424.50
1,1/2/2019,4500,3480.36
2,1/3/2019,4500,1576.38
3,1/4/2019,4500,2928.55
4,1/5/2019,4500,4055.30


In [23]:
# Read online sales data
df_onlineSales.head()

,CustomerID,Transaction_ID,Transaction_Date,Product_SKU,Product_Description,Product_Category,Quantity,Avg_Price,Delivery_Charges,Coupon_Status
0,17850,16679,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.5,Used
1,17850,16680,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.5,Used
2,17850,16681,1/1/2019,GGOEGFKQ020399,Google Laptop and Cell Phone Stickers,Office,1,2.05,6.5,Used
3,17850,16682,1/1/2019,GGOEGAAB010516,Google Men's 100% Cotton Short Sleeve Hero Tee...,Apparel,5,17.53,6.5,Not Used
4,17850,16682,1/1/2019,GGOEGBJL013999,Google Canvas Tote Natural/Navy,Bags,1,16.50,6.5,Used


In [24]:
# Read tax amount data
df_taxAmount.head()

,Product_Category,GST
0,Nest-USA,10%
1,Office,10%
2,Apparel,18%
3,Bags,18%
4,Drinkware,18%


## 💡 Business questions: